In [23]:
import mat73
import numpy as np
import pandas as pd
#from rigid_transform_3D import rigid_transform_3D

In [2]:
data_dict = mat73.loadmat('sample.mat')

In [3]:
data_dict.keys()

dict_keys(['DIC3DPPresults'])

In [4]:
data_dict['DIC3DPPresults'].keys()

dict_keys(['AllPairsResults', 'DIC2Dinfo', 'Deform', 'Deform_ARBM', 'Disp', 'FaceCentroids', 'FaceCentroids_ARBM', 'FaceColors', 'FaceCorrComb', 'FaceIsoInd', 'FacePairInds', 'Faces', 'PointPairInds', 'Points3D', 'Points3D_ARBM', 'RBM', 'calibration', 'corrComb', 'distortion', 'pairIndices'])

In [5]:
data_dict['DIC3DPPresults']['Faces'].shape

(13368, 3)

In [6]:
facesPy = data_dict['DIC3DPPresults']['Faces']

In [7]:
facesPy.shape

(13368, 3)

In [8]:
facesPy[1]
# an array with float numbers, int needed

array([4958., 4957., 4689.])

In [9]:
facesPyInt = facesPy.astype(int)
facesPyInt[1]
# got int array

array([4958, 4957, 4689])

In [10]:
#sync load file
DIC3D = data_dict['DIC3DPPresults']
#[todo] save results

In [11]:
#get number of images
nImages = len(DIC3D['Points3D']) # DIC3D['Points3D'] is a list
nImages

5

In [12]:
#DIC3D['Points3D'] is a list of 5 arrays
#DIC3D['Points3D'][0] is a list of 1 array, stand for a image
#DIC3D['Points3D'][0][0] is a list sized 6978 in this case
len(DIC3D['Points3D'][0][0])

6978

In [13]:
#pre-allocate 3D-DIC result variables
Points3D_ARBM=[]
Disp_DispVec=[]
Disp_DispMgn=[]
Disp_DispVec_ARBM=[]
Disp_DispMgn_ARBM=[]
FaceCentroids=[]
FaceCentroids_ARBM=[]
FaceCorrComb=[]
FaceIsoInd=[]
RBM_RotMat=[]
RBM_TransVec=[]

In [14]:
def rigid_transform_3D(argA, argB):
    orgA = argA# backup for data after transformation
    nanIndexa = np.argwhere(np.isnan(argA))[:,0]
    nanIndexb = np.argwhere(np.isnan(argB))[:,0]
    nanIndexab = np.concatenate([nanIndexa,nanIndexb])
    argA = np.delete(argA, nanIndexab,0)
    argB = np.delete(argB, nanIndexab,0)
    argA = argA.transpose()  
    argB = argB.transpose()
    A = np.asmatrix(argA)
    B = np.asmatrix(argB)#as required by external file
    
    # find mean column wise different approach between matlab script
    centroid_A = np.mean(A, axis=1)
    centroid_B = np.mean(B, axis=1)

    # ensure centroids are 3x1 (necessary when A or B are
    # numpy arrays instead of numpy matrices)
    centroid_A = centroid_A.reshape(-1, 1)
    centroid_B = centroid_B.reshape(-1, 1)

    # subtract mean
    Am = A - centroid_A
    Bm = B - centroid_B

    H = np.dot(Bm,np.transpose(Am))

    # find rotation
    U, S, Vt = np.linalg.svd(H)
    S_mat = [[1,0,0],[0,1,0],[0,0,np.linalg.det(U*Vt)]]
    R = Vt.T * U.T * S_mat
    R = R.T

    t = -R*centroid_A + centroid_B
    P = ((orgA * R.T).T+t).T

    return R, t, P

In [15]:
#iface in range
facesPy.shape[0]

13368

In [16]:
#Calculating displacements and rigid body motion
FaceCorrComb=[] #init, or it will append with no limit
FaceCentroids=[None]*nImages # declare it with size
Disp_DispVec=[]
Disp_DispMgn=[]
RBM_RotMat=[]
RBM_TransVec=[]
Points3D_ARBM=[]
Disp_DispVec_ARBM=[]
Disp_DispMgn_ARBM=[]
FaceCentroids_ARBM=[None]*nImages

for image in range(nImages):
    #Face correlation coefficient (worst)
    FaceCorrComb.append(np.amax(DIC3D['corrComb'][image][0][facesPyInt-1],1))# find max in each row
    #Compute face centroids
    means = []
    for ifaces in range(facesPy.shape[0]):# in shorted version will be 10, full will be 13368  
        meanOfEachLine = data_dict['DIC3DPPresults']['Points3D'][image][0][facesPyInt[ifaces,:]-1].mean(axis = 0)
        means.append(meanOfEachLine)
        FaceCentroids[image] = means
    #Compute displacements between frames (per point)
    DispVec = data_dict['DIC3DPPresults']['Points3D'][image][0] - data_dict['DIC3DPPresults']['Points3D'][0][0]
    Disp_DispVec.append(DispVec)
    Disp_DispMgn.append(np.sqrt(DispVec[:,0]**2 + DispVec[:,1]**2 + DispVec[:,2]**2))
    
    #Compute rigid body transformation between point clouds
    refP= data_dict['DIC3DPPresults']['Points3D'][image][0]
    truP = data_dict['DIC3DPPresults']['Points3D'][0][0]  
    RotMat,TransVec,P = rigid_transform_3D(refP,truP)
    RBM_RotMat.append(np.asarray(RotMat))
    RBM_TransVec.append(np.asarray(TransVec))
    Points3D_ARBM.append(np.asarray(P))
    
    #Compute displacements between sets - after RBM
    DispVecARBM = Points3D_ARBM[image] - Points3D_ARBM[0]
    #DispVecARBM = np.asarray(DispVecARBM)## check data type, matrix can not be used for **2 directly
    Disp_DispVec_ARBM.append(DispVecARBM)
    Disp_DispMgn_ARBM.append(np.sqrt(DispVecARBM[:,0]**2 + DispVecARBM[:,1]**2 + DispVecARBM[:,2]**2))
    
    #Compute face centroids - after RBM
    # NOTE Points3D_ARBM[image] is a matrix,"data_dict['DIC3DPPresults']['Points3D'][0][0]" is an array, at same shape
    meansRBM = []
    for ifaces in range(facesPy.shape[0]):# in shorted version will be 10, full will be 13368  
        meanOfEachLine = Points3D_ARBM[image][facesPyInt[ifaces,:]-1].mean(axis = 0)
        meansRBM.append(meanOfEachLine)
        FaceCentroids_ARBM[image] = meansRBM
# end of displacements and rigid body motion

In [17]:
##################################test##################################

In [26]:
FaceCentroids[1]

[array([  5.22066441,  18.58479946, 171.91580897]),
 array([  9.18385012,  47.89696964, 163.92395046]),
 array([ 18.50686774,  45.69887389, 176.85060527]),
 array([ -1.1422473 ,  20.6308122 , 199.75132499]),
 array([ 18.20015333,  45.73941915, 147.70545183]),
 array([13.21775928, 30.3323449 , 65.57307734]),
 array([11.25275021, 47.82709146, 96.10066083]),
 array([ -1.21692676,  20.74870713, 210.9644209 ]),
 array([  6.59464554,  17.58439408, 229.25120771]),
 array([  5.92587769,  17.95079372, 186.08458981]),
 array([  3.46188319,  18.89888062, 209.03902728]),
 array([-10.81401111,  24.44953848, 204.39576481]),
 array([ 12.29565024,  47.32685765, 176.35009047]),
 array([  2.81983993,  48.92750423, 129.05881774]),
 array([  1.77733713,  19.47162322, 204.5929247 ]),
 array([  4.45490497,  18.30164853, 198.89016408]),
 array([  3.40404551,  48.76907956, 168.75771846]),
 array([ 18.49077894,  45.66624129, 166.36064438]),
 array([  6.3568768 ,  48.39467493, 142.84297489]),
 array([ 22.732379

In [39]:
arr = np.array(FaceCentroids)
pd.DataFrame.to_xarray(arr)


AttributeError: 'numpy.ndarray' object has no attribute '_AXIS_ORDERS'

In [34]:
stacked = pd.Panel(arr.swapaxes(1,2)).to_frame().stack().reset_index()
stacked.columns = ['x', 'y', 'z', 'value']
# save to disk
stacked.to_csv('stacked.csv', index=False)

C:\Users\17895\Anaconda3\envs\PASS5508\lib\site-packages\IPython\core\interactiveshell.py:2882: FutureWarning: 
Panel is deprecated and will be removed in a future version.
The recommended way to represent these types of 3-dimensional data are with a MultiIndex on a DataFrame, via the Panel.to_frame() method
Alternatively, you can use the xarray package http://xarray.pydata.org/en/stable/.
Pandas provides a `.to_xarray()` method to help automate this conversion.

  exec(code_obj, self.user_global_ns, self.user_ns)


In [19]:
FaceCentroids_ARBM[1]

[array([  2.81926861,  11.41924089, 169.94342866]),
 array([  5.57064723,  41.17194495, 163.16209536]),
 array([ 14.80152328,  38.88003582, 176.13848804]),
 array([ -4.01776834,  12.076862  , 197.7339167 ]),
 array([ 14.90075082,  40.06087372, 147.01579135]),
 array([11.76539506, 27.71178033, 64.27765889]),
 array([ 8.58850874, 43.87951902, 95.42845969]),
 array([ -4.25442049,  11.74725372, 208.94053753]),
 array([  3.43524132,   8.21147108, 227.21093579]),
 array([  3.35416216,  10.25667717, 184.08612903]),
 array([  0.52935403,  10.18476673, 207.0184561 ]),
 array([-13.91556553,  15.2753201 , 202.37072415]),
 array([  8.53093938,  40.24905826, 175.60404994]),
 array([ -0.34528307,  43.29844054, 128.26768475]),
 array([ -1.11694255,  10.8577104 , 202.57206296]),
 array([  1.69002259,  10.03454101, 196.87092028]),
 array([ -0.30955576,  41.59438736, 167.9345984 ]),
 array([ 14.93356215,  39.26209135, 165.65625533]),
 array([  3.0191116 ,  42.37788892, 142.07460179]),
 array([ 19.285378

In [80]:
test = [None]*5
test[1] = [None]*10
test[1][1] = [0]*3
test

[None,
 [None, [0, 0, 0], None, None, None, None, None, None, None, None],
 None,
 None,
 None]

In [15]:
#########################################################################
#############################draft#############################

In [123]:
# look at sturcture of Cmat
print(len(DIC3D['Deform']['Cmat']),type(DIC3D['Deform']['Cmat']))
print(len(DIC3D['Deform']['Cmat'][1]),type(DIC3D['Deform']['Cmat'][1]))
print(len(DIC3D['Deform']['Cmat'][1][0]),type(DIC3D['Deform']['Cmat'][1][0]),DIC3D['Deform']['Cmat'][1][0].shape)
DIC3D['Deform']['Cmat'][1][0]

5 <class 'list'>
1 <class 'list'>
3 <class 'numpy.ndarray'> (3, 3, 13368)


array([[[ 9.54706166e-01,  9.99721181e-01,  9.99791127e-01, ...,
          9.61583530e-01,  9.64104144e-01,  9.29239937e-01],
        [ 1.25426801e-02,  4.98027651e-05,  6.45146625e-05, ...,
          2.45581977e-02,  2.29515234e-02,  4.21250553e-02],
        [-4.23826665e-02, -5.61566929e-05,  6.04123520e-05, ...,
         -9.38330928e-02, -6.44587177e-02, -5.24051118e-02]],

       [[ 1.25426801e-02,  4.98027651e-05,  6.45146625e-05, ...,
          2.45581977e-02,  2.29515234e-02,  4.21250553e-02],
        [ 9.96375337e-01,  9.99991099e-01,  9.99980063e-01, ...,
          9.84930135e-01,  9.86086963e-01,  9.73878446e-01],
        [ 9.77107333e-03,  8.55981086e-06, -2.11028149e-05, ...,
          5.00918586e-02,  3.27467305e-02,  3.80296299e-02]],

       [[-4.23826665e-02, -5.61566929e-05,  6.04123520e-05, ...,
         -9.38330928e-02, -6.44587177e-02, -5.24051118e-02],
        [ 9.77107333e-03,  8.55981086e-06, -2.11028149e-05, ...,
          5.00918586e-02,  3.27467305e-02,  3.802

In [16]:
#Compute rigid body transformation between point clouds
#error message: matrix A is not 3xN, it is 6978x3
[RotMat,TransVec] = rigid_transform_3D(
                    data_dict['DIC3DPPresults']['Points3D'][1][0],
                    data_dict['DIC3DPPresults']['Points3D'][0][0])

Exception: matrix A is not 3xN, it is 6978x3

In [146]:
#change row & column
#delete nan before transpose
argA = data_dict['DIC3DPPresults']['Points3D'][2][0]
nanIndex = np.argwhere(np.isnan(argA))[:,0]
#argA = np.delete(argA, nanIndex,0)
argA = argA.transpose()
argB = data_dict['DIC3DPPresults']['Points3D'][0][0]
nanIndex = np.argwhere(np.isnan(argB))[:,0]
#argB = np.delete(argB, nanIndex,0)
argB = argB.transpose()
print("shape of a is:",argA.shape, "b:",argB.shape)
print(type(argA))
argBm = np.asmatrix(argB)
print(type(argBm))

shape of a is: (3, 6978) b: (3, 6978)
<class 'numpy.ndarray'>
<class 'numpy.matrix'>


In [159]:
x = data_dict['DIC3DPPresults']['Points3D'][0][0]
nanIndex =  (np.isnan(x))[:,0]
x[nanIndex]

array([], shape=(0, 3), dtype=float64)

In [131]:
# nan check
x =data_dict['DIC3DPPresults']['Points3D'][4][0]
nanIndex = np.argwhere(np.isnan(x))[:,0]
x1 = np.delete(x, nanIndex,0)
x1[np.argwhere(np.isnan(x1))[:,0]]

array([], shape=(0, 3), dtype=float64)

In [68]:
# try run rigid_t here------------------------
# find mean column wise
centroid_A = np.mean(argA, axis=1)#
centroid_B = np.mean(argBm, axis=1)#array([[  0.64130487], [ 25.41937206], [119.06233202]]) 3*1 shape

In [69]:
print("shape of a is:",centroid_A.shape, "b:",centroid_B.shape)
print(centroid_A)
print(centroid_B)

shape of a is: (3,) b: (3, 1)
[  0.64130487  25.41937206 119.06233202]
[[  0.64130487]
 [ 25.41937206]
 [119.06233202]]


In [70]:
# ensure centroids are 3x1 (necessary when A or B are
# numpy arrays instead of numpy matrices)
centroid_A = centroid_A.reshape(-1, 1) # to 3*1 shape
centroid_B = centroid_B.reshape(-1, 1)

In [71]:
num_rows, num_cols = argA.shape;
# subtract mean
Am = argA - np.tile(centroid_A, (1, num_cols))
Bm = argB - np.tile(centroid_B, (1, num_cols))

In [55]:
H = np.dot(Am,np.transpose(Bm))

In [56]:
# find rotation
U, S, Vt = np.linalg.svd(H)
R = Vt.T * U.T

In [57]:
t = -R*centroid_A + centroid_B


In [58]:
R

matrix([[ 0.99888906, -0.0450026 , -0.0139792 ],
        [ 0.04441871,  0.99822806, -0.03959452],
        [ 0.01573628,  0.03892959,  0.99911804]])

In [59]:
Vt
#rigid_t ends here---------------------------------

matrix([[-0.04064423, -0.06882704, -0.99680032],
        [-0.54752799, -0.83296977,  0.07984016],
        [-0.8357997 ,  0.54902112, -0.00382932]])

In [91]:
RotMat,TransVec = rigid_transform_3D(argA,argBm)

LinAlgError: SVD did not converge

In [73]:
print(RotMat)
print(TransVec)

[[ 1.00000000e+00  7.12971599e-17 -8.10578730e-17]
 [-6.38875587e-17  1.00000000e+00  3.43774206e-16]
 [-5.50102670e-17  6.17902485e-17  1.00000000e+00]]
[[ 7.99360578e-15]
 [-3.55271368e-14]
 [-2.84217094e-14]]


In [28]:
#Compute displacements between frames (per point)
DispVec = data_dict['DIC3DPPresults']['Points3D'][1][0] - data_dict['DIC3DPPresults']['Points3D'][0][0] 

In [29]:
# this produce same error
# done debugging
def testfunction(A,B):
    #if len(A) == len(B):
        #print("wellllll")
   # else:
       # print("wrong")
    print(A.shape)
    print(len(B))
testfunction(argA,argB)


(3, 6978)
3


In [30]:
np.sqrt(DispVec[:,0]**2 + DispVec[:,1]**2 + DispVec[:,2]**2)[0:10]

array([21.12711363, 21.12998768, 21.18308559, 21.30089595, 21.31974562,
       21.27991817, 21.33715132, 21.30546194, 21.24607238, 21.31043549])

In [31]:
#compute face centroids
facesPyInt.shape
facesPyInt[1,:]

array([4958, 4957, 4689])

In [32]:
data_dict['DIC3DPPresults']['Points3D'][1][0][facesPyInt[1,:]-1]# frist 1: image, second 1: face(to 13368)

array([[  8.87010205,  47.95226179, 163.64820456],
       [  8.87154231,  47.95528282, 164.5318443 ],
       [  9.80990601,  47.78336433, 163.59180251]])

In [33]:
meanOfEachLine = data_dict['DIC3DPPresults']['Points3D'][1][0][facesPyInt[1,:]-1].mean(axis = 0) 
meanOfEachLine

array([  9.18385012,  47.89696964, 163.92395046])

In [34]:
#read coorComb
DIC3D['corrComb'][0][0][facesPyInt-1] #DIC3D.corrComb{ii}(F); in this case we have 10 instances

array([[1.35222278, 1.36593578, 1.29955775],
       [0.16628634, 0.16344256, 0.19751639],
       [0.16086167, 0.13946344, 0.13863298],
       [1.25526403, 1.25750794, 1.28705816],
       [0.15405274, 0.17574434, 0.1730297 ],
       [0.06320487, 1.52480727, 0.0826026 ],
       [0.21619105, 0.21821001, 0.21504012],
       [1.27900074, 1.26905087, 1.28909456],
       [1.49670128, 1.46146731, 1.46961375],
       [1.27959089, 1.23319027, 1.24494115]])

In [35]:
x = np.amax(DIC3D['corrComb'][1][0][facesPyInt],1) # find max in each row
x

array([1.10297569, 0.20416364, 0.16270319, 1.30087252, 0.18329407,
       1.27172527, 0.22074969, 1.27464799, 1.41727779, 1.29572459])

In [36]:
facesPy

array([[ 610.,  338.,  339.],
       [4958., 4957., 4689.],
       [2248., 1977., 1978.],
       [2210., 2211., 2480.],
       [2011., 2282., 2281.],
       [1021., 1022., 1293.],
       [4496., 4228., 4497.],
       [2469., 2468., 2199.],
       [   7.,  279.,  278.],
       [ 323.,   52.,  324.]])